<a href="https://colab.research.google.com/github/thaianh1210/DecisionTree/blob/main/Decision_Tree_FromScratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import numpy as np
from collections import Counter
from sklearn.metrics import accuracy_score

In [34]:
#Calculate entropy
def entropy(s):
  count = np.bincount(s)
  print(count)
  percentages = count / len(s)
  entropy = 0
  for percentage in percentages:
    if percentage > 0:
      entropy += (percentage * np.log2(percentage))
  return -entropy


In [35]:
#Testcase
s = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
case = np.round(entropy(s), 3)
case

[7 3]


0.881

In [36]:
#Calculate information gain
def information_gain(parent, left_child, right_child):
  num_left = len(left_child) / len(parent)
  num_right = len(right_child) / len(parent)
  gain = entropy(parent) - (num_left * entropy(left_child) + num_right * entropy(right_child))
  return gain


In [37]:
#Testcase
parent = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]
left_child = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
right_child = [0, 0, 0, 0, 1, 1, 1, 1]
print(np.round(information_gain(parent, left_child, right_child), 3))

[12  8]
[10  2]
[4 4]
0.181


In [38]:
class TreeNode():
  def __init__(self, feature = None, threshold = None, data_left = None, data_right = None, gain = None, value = None):
    self.feature = feature
    self.threshold = threshold
    self.data_left = data_left
    self.data_right = data_right
    self.gain = gain
    self.value = value


In [47]:
class DecisionTree():
  def __init__(self, min_samples_split = 2, max_depth = 5):
    self.min_samples_split = min_samples_split
    self.max_depth = max_depth
    self.root = None

  @staticmethod
  #Calculate entropy
  def _entropy(s):
    #Convert to intergers to avoid runtime error
    count = np.bincount(np.array(s, dtype = np.int64))
    #Calculate the probability of each class label
    percentages = count / len(s)
    entropy = 0
    for percentage in percentages:
      if percentage > 0:
        entropy += (percentage * np.log2(percentage))
    return -entropy

  def information_gain(self, parent, left_child, right_child):
    '''
    Helper function, calculates information gain from a parent and two child nodes.

      :param parent: list, the parent node
      :param left_child: list, left child of a parent
      :param right_child: list, right child of a parent
      :return: float, information gain
    '''
    num_left = len(left_child) / len(parent)
    num_right = len(right_child) / len(parent)
    gain = self._entropy(parent) - (num_left * self._entropy(left_child) + num_right * self._entropy(right_child))
    return gain

  def best_split(self, X, y):
    '''
    Helper function, calculates the best split for given features and target

      :param X: np.array, features
      :param y: np.array or list, target
      :return: dict
    '''
    best_split = {}
    best_info_gain = -1
    n_rows, n_cols = X.shape

    #For every dataset features:
    for f_idx in range(n_cols):
      X_curr = X[:, f_idx]
      #For every unique value of the feature:
      for threshold in np.unique(X_curr):
        # Construct a dataset and split it to the left and right parts
        # Left part includes records lower or equal to the threshold
        # Right part includes records higher than the threshold
        df = np.concatenate((X, y.reshape(1, -1).T), axis = 1)
        df_left = np.array([row for row in df if row[f_idx] <= threshold])
        df_right = np.array([row for row in df if row[f_idx] > threshold])

        if len(df_left) > 0 and len(df_right) > 0:
          y = df[:, -1]
          y_left = df_left[:, -1]
          y_right = df_right[:, -1]

        gain = self.information_gain(y, y_left, y_right)
        if gain > best_info_gain:
          best_split = {
              'feature_index':f_idx,
              'threshold': threshold,
              'df_left': df_left,
              'df_right': df_right,
              'gain': gain
          }
          best_info_gain = gain
    return best_split

  def _build(self, X, y, depth = 0):

    '''
      Helper recursive function, used to build a decision tree from the input data.

      :param X: np.array, features
      :param y: np.array or list, target
      :param depth: current depth of a tree, used as a stopping criteria
      :return: Node
      '''
    n_rows, n_cols = X.shape
    if n_rows >= self.min_samples_split and depth <= self.max_depth:
      #Get the best split
      best = self.best_split(X, y)
      if best['gain'] > 0:
        #Build a tree on the left
        left = self._build(
            X = best['df_left'][:, :-1],
            y = best['df_left'][:, -1]
        )
        right = self._build(
            X = best['df_right'][:, :-1],
            y = best['df_right'][:, -1]
        )
        return TreeNode(
            feature = best['feature_index'],
            threshold = best['threshold'],
            data_left = left,
            data_right = right,
            gain = best['gain']
        )
    return TreeNode(
        value=Counter(y).most_common(1)[0][0]
        )
  def fit(self, X, y):
      self.root = self._build(X, y)

  def _predict(self, x, tree):
    if tree.value != None:
      return tree.value
    feature_value = x[tree.feature]

    if feature_value <= tree.threshold:
      return self._predict(x=x, tree = tree.data_left)

    if feature_value > tree.threshold:
      return self._predict(x=x, tree = tree.data_right)

  def predict(self, X):
    return [self._predict(x, self.root) for x in X]


In [40]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

iris = load_iris()
X = iris['data']
y = iris['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [48]:
model = DecisionTree()
model.fit(X_train, y_train)
preds = model.predict(X_test)
accuracy_score(y_test, preds)

1.0